# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kerikeri,-35.2268,173.9474,19.39,62,0,0.45,NZ,1711752512
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,-5.67,58,0,2.00,RU,1711752433
2,2,nagqu,31.5000,92.0000,-4.18,63,46,0.77,CN,1711752514
3,3,tangalle,6.0240,80.7911,24.89,85,27,2.64,LK,1711752516
4,4,bodo,67.2800,14.4050,2.10,68,75,10.80,NO,1711752292


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    frame_height=450
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df=city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
wind_df=temp_df.loc[temp_df["Wind Speed"]<4.5]
narrow_df=wind_df.loc[wind_df["Cloudiness"]==0]

# Drop any rows with null values
narrow_df.dropna()

# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,71,greenville,35.6127,-77.3663,21.09,35,0,3.60,US,1711752620
126,126,galliano,29.4422,-90.2992,22.78,40,0,3.60,US,1711752702
160,160,pathein,16.7833,94.7333,22.48,82,0,2.08,MM,1711752753
168,168,kone,-21.0595,164.8658,26.34,56,0,3.29,NC,1711752765
229,229,crucecita,-28.9500,-58.9833,24.11,71,0,2.86,AR,1711752857
231,231,nueva gerona,21.8833,-82.8000,26.63,47,0,4.20,CU,1711752860
241,241,saint-pierre,-21.3393,55.4781,25.82,73,0,2.06,RE,1711752876
247,247,cam ranh,11.9214,109.1591,24.98,83,0,0.51,VN,1711752885
268,268,tallulah,32.4085,-91.1868,26.00,24,0,2.06,US,1711752916
506,506,hassi messaoud,31.6804,6.0729,22.92,9,0,3.09,DZ,1711753288


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrow_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
71,greenville,US,35.6127,-77.3663,35,
126,galliano,US,29.4422,-90.2992,40,
160,pathein,MM,16.7833,94.7333,82,
168,kone,NC,-21.0595,164.8658,56,
229,crucecita,AR,-28.9500,-58.9833,71,
231,nueva gerona,CU,21.8833,-82.8000,47,
241,saint-pierre,RE,-21.3393,55.4781,73,
247,cam ranh,VN,11.9214,109.1591,83,
268,tallulah,US,32.4085,-91.1868,24,
506,hassi messaoud,DZ,31.6804,6.0729,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,"Lat"]
    longitude=hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    params['categories']=f"accommodation.hotel"


    # Make and API request using the params dictionaty

    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
greenville - nearest hotel: Quality Inn
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
pathein - nearest hotel: Taan Taan Ta
kone - nearest hotel: Pacifik Appartels
crucecita - nearest hotel: No hotel found
nueva gerona - nearest hotel: Hotel Bamboo
saint-pierre - nearest hotel: Tropic Hotel
cam ranh - nearest hotel: Khách sạn Thái Bình
tallulah - nearest hotel: No hotel found
hassi messaoud - nearest hotel: AMC
sur - nearest hotel: Sur Hotel
beacon square - nearest hotel: Fairfield Inn & Suites by Marriott Holiday Tarpon Springs
broken hill - nearest hotel: Ibis Styles
corral de bustos - nearest hotel: Hotel Jaureguialzo


,City,Country,Lat,Lng,Humidity,Hotel Name
71,greenville,US,35.6127,-77.3663,35,Quality Inn
126,galliano,US,29.4422,-90.2992,40,Fairfield Inn & Suites by Marriott Cut Off-Gal...
160,pathein,MM,16.7833,94.7333,82,Taan Taan Ta
168,kone,NC,-21.0595,164.8658,56,Pacifik Appartels
229,crucecita,AR,-28.9500,-58.9833,71,No hotel found
231,nueva gerona,CU,21.8833,-82.8000,47,Hotel Bamboo
241,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
247,cam ranh,VN,11.9214,109.1591,83,Khách sạn Thái Bình
268,tallulah,US,32.4085,-91.1868,24,No hotel found
506,hassi messaoud,DZ,31.6804,6.0729,9,AMC


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    frame_height=450,
    hover_cols=["Lng","Lat","City","Humidity","Hotel Name","Country"]
    
    
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)